In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 

In [ ]:
df = pd.read_csv('Hitters.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.isnull().sum()

In [ ]:
df['Salary']

In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
imputer = KNNImputer(n_neighbors = 3)

In [ ]:
df['Salary'] = imputer.fit_transform(df['Salary'].values.reshape(-1,1))

In [ ]:
df['Salary']

In [ ]:
df.isnull().sum()

In [ ]:
df_num = df.select_dtypes(include=['float','int'])
df_num.info()

In [ ]:
#BOXPLOT İLE AYRIK DEĞER ANALİZİ YAPTIK

In [ ]:
for i in df_num.columns:
  sns.boxplot(x = i, data=df)
  plt.show()

#HER BİR SÜTUNUN AYRIK DEĞERLERİNİ BASKILAMA YÖNTEMİ KULLANARAK KALDIRDIK.

In [ ]:
for i in df_num.columns:
  q1 = df[i].quantile(0.25)
  q3 = df[i].quantile(0.75)
  IQR = q3 - q1 
  alt_sinir = q1 - 1.5 * IQR
  ust_sinir = q3 + 1.5 * IQR
  df[i][(df[i] > ust_sinir)] = ust_sinir
  df[i][(df[i] < alt_sinir)] = alt_sinir

In [ ]:
for i in df_num.columns:
  sns.boxplot(x = i, data=df)
  plt.show()

In [ ]:
sns.pairplot(df)

In [ ]:
for i in df_num.columns:
  sns.scatterplot(df_num['Salary'], df[i])
  plt.show()

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(df.corr(), annot=True)

In [ ]:
x = df.drop('Salary', axis=1)
y = df['Salary']
x_encoded = pd.get_dummies(x, prefix_sep='_')
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
y_encoded = LabelEncoder().fit_transform(y)
x_scaled = StandardScaler().fit_transform(x_encoded)

In [ ]:
#Hedef değişken
target = abs(df.corr()["Salary"])
target

In [ ]:
#En yüksekleri seçiyoruz.
target.sort_values(ascending=False)
target = target[target>0.4]
target.head()

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_scaled,y_encoded,test_size=0.30,random_state=0)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,cross_val_score
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error


In [ ]:

models = [("lr",LinearRegression()),
          ("knn",KNeighborsRegressor(n_neighbors=10)),
          ("dt",DecisionTreeRegressor()),
          ("nb",GaussianNB()),
          ("svr",SVR()),
          ("rf",RandomForestRegressor(n_estimators=1000)),
          ("xgb",XGBRegressor(max_depth=1000,learning_rate=0.1, n_estimators=100))]


sonuclar = []
isimler = []
for name, model in models:
    model.fit(x_train, y_train)
    tahmin  = model.predict(x_test)
    sonuc = np.sqrt(mean_squared_error(y_test, tahmin))
    sonuclar.append(sonuc)
    isimler.append(name)
    print(f"{name}: {np.mean(sonuclar)}")